In [1]:
import os
import sys
import scrapping
import pandas as pd

usa_cities = pd.read_csv("../files/data/usa_cities.csv")
usa_states = pd.read_csv("../files/data/usa_states.csv")

In [2]:
#
import re
import time
import pandas as pd

#
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service as ChromeService

# This function takes a list of hotel's urls and returns a panads dataframe with all the reviews
def scrape_hotels_reviews(urls: list, report=True, interfase=False) -> list:
    
    hotel_reviews = pd.DataFrame(columns=["nationality", "acommodation", "stay", "company", "review_title", "good_review", "bad_review"])

    chrome_options = selenium.webdriver.chrome.options.Options()
    chrome_options.add_argument('--headless') if not interfase else None
    chrome_options.add_argument('--disable-infobars') # disables image loading
    driver = webdriver.Chrome(options=chrome_options) 

    for i in range(len(urls)):


        print(f"{i+1}/{len(urls)}")
        driver.get(urls[i]) # acces to url
        time.sleep(5) # wait to load
        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight*0.45);")
        buttons = driver.find_elements(By.CLASS_NAME, 'a83ed08757')
        for attempt in range(3):
            try:
                for b in buttons:
                    if b.text == "Leer todos los comentarios":
                        b.click()
                        break
            except StaleElementReferenceException:
                continue  # reintenta si se produce la excepción
            else:
                break  # sal del bucle si la operación fue exitosa

        time.sleep(2) # wait to load
        pagination = driver.find_element(By.CLASS_NAME, "c-pagination")
        pages = pagination.find_elements(By.CLASS_NAME, "bui-pagination__item")
        n_p = pages[-2].text
        parts = n_p.split(" ")
        n_pages = int(parts[1])

        for page in range(n_pages):
            time.sleep(2)
            review_list = driver.find_element(By.CLASS_NAME, 'review_list')
            review_blocks = review_list.find_elements(By.CLASS_NAME, 'review_list_new_item_block')
            for block in review_blocks:

                try: # obtein user information
                    user_information_element = block.find_elements(By.CLASS_NAME, "bui-list__item")
                    if len(user_information_element) == 3:
                        acommodation = user_information_element[0].text
                        print(acommodation) if report else None
                        try:
                            stay, date = user_information_element[1].text.split(' ·  ')
                            print(stay) if report else None
                            print(date) if report else None
                        except Exception as e:
                            stay = None
                            date = None
                        company = user_information_element[2].text
                        print(company) if report else None
                    elif len(user_information_element) == 2:
                        try:
                            stay, date = user_information_element[0].text.split(' ·  ')
                            print(stay) if report else None
                            print(date) if report else None
                        except Exception as e:
                            stay = None
                            date = None
                        company = user_information_element[1].text
                        print(company) if report else None
                    else:
                        acommodation = None
                        stay = None
                        date = None
                        company = None
                        print(f"ERROR WHILE SCRAPPING USER INFORMATION: {len(user_information_element)} elements founded, excepction must be handled")
                        for element in user_information_element:
                            print(element.text)
                        print("-------------------------------------------------------------------------------------------------------------------")
                except Exception as e:
                    acommodation = None
                    stay = None
                    date = None
                    company = None
                    print(f"ERROR: WHILE SCRAPING USER INFORMATION - {type(e).__name__}: {e}")

                try: # obtein user nationality
                    nationality_element = block.find_elements(By.CLASS_NAME, "bui-avatar-block__subtitle")
                    nationality = nationality_element[0].text
                    print(nationality) if report else None
                except Exception as e: 
                    nationality =  None
                    print("ERROR: WHILE SCRAPPING USER NATIONALITY")

                try: # obtein review title
                    review_title_element = block.find_element(By.CLASS_NAME, "c-review-block__title")
                    review_title = review_title_element.text
                    print(review_title) if report else None
                except Exception as e:
                    review_title =  None
                    print("ERROR: WHILE SCRAPPING TITLE REVIEW")



                try: # obtein rate
                    rate_element = block.find_element(By.CLASS_NAME, "bui-review-score__badge")
                    rate = rate_element.text
                    #print(rate)
                except Exception as e:
                    rate =  None
                    print("ERROR: WHILE SCRAPPING RATE")


                try: # obtein date
                    date_element = block.find_elements(By.CLASS_NAME, "c-review-block__date")
                    complete_date = date_element[1].text
                except Exception as e:
                    complete_date =  None
                    print("ERROR: WHILE SCRAPPING DATE")

                try: # obtein positive and bad reviews
                    reviews_element = block.find_elements(By.CLASS_NAME, "c-review__body")
                    if len(reviews_element) == 1:
                        good_review =  reviews_element[0].text
                        print(good_review) if report else None
                        bad_review =  None
                    elif len(reviews_element) == 2:
                        good_review =  reviews_element[0].text
                        print(good_review) if report else None
                        bad_review =  reviews_element[1].text
                        print(bad_review) if report else None
                    elif len(reviews_element) == 4:
                        good_review =  reviews_element[0].text
                        print(good_review) if report else None
                        bad_review =  reviews_element[2].text
                        print(bad_review) if report else None
                    else:
                        good_review =  None
                        bad_review =  None
                        print(f"ERROR WHILE SCRAPPING POSITIVE AND NEGATIVE REVIEWS: {len(reviews_element)} elements founded, excepction must be handled")
                        for element in reviews_element:
                            print(element.text)
                        print("-------------------------------------------------------------------------------------------------------------------")
                except Exception as e:
                    good_review =  None
                    bad_review =  None
                    print("ERROR: WHILE SCRAPPING POSITIVE AND NEGATIVE REVIEWS")
            
                # add data to dataframe
                review = pd.DataFrame({
                    "hotel_id": [urls[i]],
                    "nationality": [nationality],
                    "acommodation": [acommodation],
                    "stay": [stay],
                    "date": [date],
                    "company": [company],
                    "rate": [rate],
                    "review_title": [review_title],
                    "good_review": [good_review],
                    "bad_review": [bad_review],
                    "complete_date": [complete_date]})
                hotel_reviews = pd.concat([hotel_reviews, review], ignore_index=True)

            # Go to next page
            if page+1 < n_pages:
                next_page_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CLASS_NAME, 'pagenext'))) 
                driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)
                time.sleep(1)
                next_page_button.click() 

    driver.quit()
    return hotel_reviews




### Hyatt

California

In [3]:
hotels = pd.read_csv("../files/data/booking/booking_hotels.csv", index_col=0)
hotels.dropna(subset=["name"], inplace=True)
clients_hotels_california = hotels[(hotels["name"].str.contains("Hyatt")) & (hotels["state"] == "California")]
clients_hotels_california

,state,city,name,avg_score,price,reviews_url,stars,hotel_id
4182,California,Mission Viejo,Hyatt House Irvine/John Wayne Airport,88.0,363.318,https://www.booking.com/hotel/us/hyatt-house-i...,3.0,12620
4006,California,San Jose,Hyatt Place San Jose Pinares,87.0,262.134,https://www.booking.com/hotel/cr/hyatt-place-s...,3.0,13999
3398,California,San Francisco,Hyatt Regency San Francisco,87.0,806.063,https://www.booking.com/hotel/us/hyatt-regency...,4.0,14468
3564,California,San Diego,Manchester Grand Hyatt San Diego,86.0,620.385,https://www.booking.com/hotel/us/manchester-gr...,4.0,16129
3500,California,San Francisco,Hyatt Regency San Francisco Downtown SOMA,86.0,1020.140,https://www.booking.com/hotel/us/hyatt-regency...,4.0,16172
3589,California,San Diego,Manchester Grand Hyatt San Diego,86.0,620.385,https://www.booking.com/hotel/us/manchester-gr...,4.0,16325
3442,California,San Francisco,Hyatt Place San Francisco/Downtown,85.0,631.307,https://www.booking.com/hotel/us/hyatt-place-s...,3.0,17919
3214,California,Los Angeles,"Thompson Hollywood, part of Hyatt",85.0,657.302,https://www.booking.com/hotel/us/thompson-holl...,4.0,18809
3196,California,Los Angeles,Hyatt House LA - University Medical Center,84.0,568.180,https://www.booking.com/hotel/us/hyatt-house-l...,3.0,18945
3574,California,San Diego,Hyatt Regency La Jolla at Aventine,84.0,443.444,https://www.booking.com/hotel/us/hyatt-regency...,4.0,20143


In [4]:
reviews_california = scrape_hotels_reviews(clients_hotels_california["reviews_url"].tolist(), report=False, interfase=True)
reviews_california

1/24
2/24
3/24
ERROR: WHILE SCRAPPING TITLE REVIEW
ERROR: WHILE SCRAPPING TITLE REVIEW
ERROR: WHILE SCRAPPING USER NATIONALITY
ERROR: WHILE SCRAPPING USER NATIONALITY
4/24
ERROR: WHILE SCRAPPING TITLE REVIEW
ERROR: WHILE SCRAPPING USER NATIONALITY
ERROR: WHILE SCRAPPING TITLE REVIEW
ERROR: WHILE SCRAPPING TITLE REVIEW
ERROR: WHILE SCRAPPING USER NATIONALITY
5/24
ERROR: WHILE SCRAPPING TITLE REVIEW
ERROR: WHILE SCRAPPING TITLE REVIEW
ERROR: WHILE SCRAPPING TITLE REVIEW
ERROR: WHILE SCRAPPING POSITIVE AND NEGATIVE REVIEWS
ERROR: WHILE SCRAPING USER INFORMATION - StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=119.0.6045.200); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF621C082B2+55298]
	(No symbol) [0x00007FF621B75E02]
	(No symbol) [0x00007FF621A305AB]
	(No symbol) [0x00007FF

,nationality,acommodation,stay,company,review_title,good_review,bad_review,hotel_id,date,rate,complete_date
0,Argentina,Suite Estudio con cocina y sofá cama - Cama ex...,4 noches,En grupo,buen hotel de 3 estrellas,"el hotel era cómodo para familias, el gimnasio...",es caro para un 4 estrellas,https://www.booking.com/hotel/us/hyatt-house-i...,diciembre de 2022,"7,0",Comentó en: 5 de enero de 2023
1,España,Den Room with King Bed,11 noches,Persona que viaja sola,sitio caro,cerca de mis sitios de interés,"limpieza, desayuno,personal",https://www.booking.com/hotel/us/hyatt-house-i...,noviembre de 2021,"5,0",Comentó en: 19 de diciembre de 2021
2,Corea del Sur,Suite Estudio con cocina y sofá cama - Cama ex...,27 noches,Persona que viaja sola,Excepcional,what a great honor to stay here.\nThank you so...,no worries.,https://www.booking.com/hotel/us/hyatt-house-i...,noviembre de 2023,10,Comentó en: 20 de noviembre de 2023
3,Taiwán,Den Room with King Bed,4 noches,Persona que viaja sola,Would go again if I had to travel for work,"Everything works and fairly new, I loved the w...",Small swimming pools,https://www.booking.com/hotel/us/hyatt-house-i...,noviembre de 2023,10,Comentó en: 15 de noviembre de 2023
4,Estados Unidos,Suite Estudio con cocina y sofá cama - Cama ex...,4 noches,En familia,Excepcional,Weather was ideal\nLoved the swimming pool.,,https://www.booking.com/hotel/us/hyatt-house-i...,noviembre de 2023,10,Comentó en: 14 de noviembre de 2023
...,...,...,...,...,...,...,...,...,...,...,...
34199,Estados Unidos,Habitación con cama extragrande y vistas al mar,2 noches,En familia,Excepcional,Esta entrada no tiene comentarios,None,https://www.booking.com/hotel/us/hyatt-regency...,julio de 2021,10,Comentó en: 31 de julio de 2021
34200,Estados Unidos,Suite con 2 camas dobles,1 noche,En familia,Excepcional,Esta entrada no tiene comentarios,None,https://www.booking.com/hotel/us/hyatt-regency...,diciembre de 2020,10,Comentó en: 2 de enero de 2021
34201,Alemania,Suite con 2 camas dobles,1 noche,En grupo,Fantástico,Esta entrada no tiene comentarios,None,https://www.booking.com/hotel/us/hyatt-regency...,mayo de 2023,"9,0",Comentó en: 2 de mayo de 2023
34202,Alemania,Suite con 2 camas dobles,2 noches,En grupo,Excepcional,Esta entrada no tiene comentarios,None,https://www.booking.com/hotel/us/hyatt-regency...,abril de 2023,10,Comentó en: 12 de mayo de 2023


In [5]:
reviews_california.to_csv('../files/data/booking/hotel_reviews/reviews_hyatt_california.csv')

### Hampton

Florida

In [6]:
hotels = pd.read_csv("../files/data/booking/booking_hotels.csv", index_col=0)
hotels.dropna(subset=["name"], inplace=True)
clients_hotels_florida = hotels[(hotels["name"].str.contains("Hampton")) & (hotels["state"] == "Florida")]
clients_hotels_florida

,state,city,name,avg_score,price,reviews_url,stars,hotel_id
8838,Florida,Kissimmee,Hampton Inn Kissimmee North,90.0,315.525,https://www.booking.com/hotel/us/hampton-inn-k...,NaN,10433
8110,Florida,Orlando,Hampton Inn & Suites Orlando-East UCF,89.0,277.371,https://www.booking.com/hotel/us/hampton-inn-s...,NaN,11087
7886,Florida,Tampa,Hampton Inn Tampa Downtown Channel District,89.0,352.054,https://www.booking.com/hotel/us/hampton-inn-t...,NaN,11319
8430,Florida,Jacksonville,Hampton Inn & Suites Jacksonville Deerwood Park,89.0,276.806,https://www.booking.com/hotel/us/hampton-inn-s...,NaN,12133
8237,Florida,Orlando,Hampton Inn & Suites Orlando-John Young Parkwa...,88.0,338.309,https://www.booking.com/hotel/us/hampton-inn-s...,NaN,12507
8470,Florida,Jacksonville,Hampton Inn & Suites Jacksonville Beach Boulev...,86.0,317.439,https://www.booking.com/hotel/us/hampton-inn-s...,NaN,15687
7965,Florida,Tampa,Hampton Inn & Suites Tampa-North,86.0,358.588,https://www.booking.com/hotel/us/hampton-inn-s...,NaN,16220
8431,Florida,Jacksonville,Hampton Inn & Suites Jacksonville South - Bart...,86.0,279.949,https://www.booking.com/hotel/us/hampton-inn-s...,NaN,16369
8788,Florida,Bonita Springs,Hampton Inn Bonita Springs Naples North,85.0,308.197,https://www.booking.com/hotel/us/hampton-inn-b...,NaN,17288
8809,Florida,Bonita Springs,Hampton Inn Bonita Springs Naples North,85.0,308.197,https://www.booking.com/hotel/us/hampton-inn-b...,NaN,18687


In [ ]:
reviews_florida = scrape_hotels_reviews(clients_hotels_florida["reviews_url"].tolist(), report=False, interfase=True)
reviews_florida

In [ ]:
reviews_florida.to_csv('../files/data/booking/hotel_reviews/reviews_hampton_florida.csv')